# PyTorch BERT for Question Answering on SageMaker

TODO: Some kind of intro

## Config & Init Stuff

In [ ]:
# For easier dev of local modules:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:

# External Dependencies:
import boto3
import sagemaker
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator


In [ ]:
BUCKET_NAME = "2020-05-gym-bert"
%store BUCKET_NAME

In [ ]:
role = sagemaker.get_execution_role()
botosess = boto3.session.Session()
region = botosess.region_name
s3 = botosess.resource("s3")
bucket = s3.Bucket(BUCKET_NAME)
smclient = botosess.client("sagemaker")

## Download data and put in S3
Download squad data from github

In [ ]:
!curl https://raw.githubusercontent.com/rajpurkar/SQuAD-explorer/master/dataset/train-v1.1.json --output ./dataset/train-v1.1.json
!curl https://raw.githubusercontent.com/rajpurkar/SQuAD-explorer/master/dataset/dev-v1.1.json --output ./dataset/dev-v1.1.json

Upload squad data to S3 bucket

In [ ]:
train_filename = 'SQuAD_train-v1.1.json'
val_filename = 'SQuAD_dev-v1.1.json'

In [ ]:
boto3.Session().resource('s3').Bucket(BUCKET_NAME).Object(train_filename).upload_file('./dataset/train-v1.1.json')
boto3.Session().resource('s3').Bucket(BUCKET_NAME).Object(val_filename).upload_file('./dataset/dev-v1.1.json')

## Configure Algorithm

Create input channels

In [ ]:
train_channel = f"s3://{BUCKET_NAME}/{train_filename}"

In [ ]:
val_channel = f"s3://{BUCKET_NAME}/{val_filename}"

Estimator

In [ ]:
estimator = PyTorchEstimator(
    entry_point="train.py",
    source_dir="src",

    base_job_name="bert-qna",
    #checkpoint_s3_uri=f"s3://{BUCKET_NAME}/checkpoints",
    output_path=f"s3://{BUCKET_NAME}/",

    framework_version="1.4.0",
    py_version="py3",

    role=role,
    train_instance_count=1,
    train_instance_type="local", # "ml.p2.xlarge",
    train_max_run=60*60,
    train_max_wait=60*60,
    #train_use_spot_instances=True,

    hyperparameters={
        "seed": 1337,
        "log-level": "DEBUG",
    },
)

## Train

In [ ]:
estimator.fit({ "train": train_channel, "val": val_channel })

## Deploy

## Clean Up

In [ ]:
...